In [1]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import load_puzzle_input, test_code
import re
import networkx as nx

In [2]:
# load puzzle input
    
puzzle_input = load_puzzle_input('input.txt')

print(puzzle_input[:3])

['Valve TN has flow rate=0; tunnels lead to valves IX, ZZ', 'Valve DS has flow rate=0; tunnels lead to valves IF, OU', 'Valve OP has flow rate=0; tunnels lead to valves UH, ZQ']


### Part 1

In [3]:
# code for preparing and testing examples for part 1

example_dict_part_1 = {
    tuple(load_puzzle_input('example_1.txt')): 1651
}

In [4]:
def parse_puzzle_input(puzzle_input):
    
    output_list = []
    
    for line in puzzle_input:
    
        matches = re.search(r"Valve (.{2}) has flow rate=(\d*); .* valve[s]* (.*)", line)
        
        output_list.append([matches.group(x) for x in range(1, 3+1)])
    
    return output_list

In [5]:
def build_valve_dict(parsed_puzzle_input):
    
    return {
        valve_name: {
            "flow": int(flow_rate_str),
            "tunnels": tunnels.split(", "),
        }
        for valve_name, flow_rate_str, tunnels in parsed_puzzle_input
    }

In [6]:
def generate_graph(valve_dict):
    
    G = nx.DiGraph()
    
    for valve_name, valve_info in valve_dict.items():
        
        G.add_node(valve_name, flow=valve_info["flow"])
        
        for tunnel in valve_info["tunnels"]:
            
            G.add_edge(valve_name, tunnel)
            
    return G

In [7]:
def find_shortest_path_length(start, finish, G):
    
    return nx.shortest_path_length(G, start, finish)

In [36]:
def decide_next_valve(start, remaining_time, G, opened_valves = set()):
    
    unopened_nodes = set(G.nodes) - opened_valves

    if not len(unopened_nodes):
        return None, 1

    next_node_value_dict = {}
    next_node_time_dict = {}

    for next_node in unopened_nodes:

        time_taken = find_shortest_path_length(start, next_node, G) + 1

        if time_taken <= remaining_time:

            next_node_value_dict[next_node] = G.nodes[next_node]["flow"] * (remaining_time - time_taken - 1)
            next_node_time_dict[next_node] = time_taken

    print(next_node_value_dict)
    print(next_node_time_dict)
    print()

    return (
        (
            max(next_node_value_dict, key=next_node_value_dict.get),
            next_node_time_dict[
                max(next_node_value_dict, key=next_node_value_dict.get)
            ],
        )
        if len(next_node_value_dict)
        else (None, 1)
    )

In [37]:
def calculate_total_flow(open_sequence, G):
    
    return sum(
        G.nodes[node]["flow"] * (30 - idx - 1)
        for idx, node in enumerate(open_sequence)
        if node
    )

In [38]:
def part_1_solution(puzzle_input):
    
    parsed_puzzle_input = parse_puzzle_input(puzzle_input)

    valve_dict = build_valve_dict(parsed_puzzle_input)

    Graph = generate_graph(valve_dict)
    
    # print(
    #     calculate_total_flow(
    #         [None, 'DD', None, None, 'BB', None, None, None, 'JJ', None, None, None, None, None, None, None, 'HH', None, None, None, 'EE', None, None, 'CC', None, None, None, None, None, None],
    #         Graph
    #     )
    # )

    opened_valves = set()
    start = "AA"
    open_sequence = []

    remaining_time = 30

    while remaining_time > 0:
        
        next_valve, time_taken = decide_next_valve(start, remaining_time, Graph, opened_valves)
        
        # print(f"next_valve: {next_valve}, time_taken: {time_taken}")
        
        open_sequence.extend(None for _ in range(time_taken))
        open_sequence.append(next_valve)
        opened_valves.add(next_valve)
        start = next_valve
        remaining_time -= time_taken

    print(open_sequence)    

    return calculate_total_flow(open_sequence, Graph)

In [39]:
# test part 1 solution

test_code(part_1_solution, example_dict_part_1)

{'JJ': 546, 'FF': 0, 'GG': 0, 'EE': 78, 'DD': 540, 'II': 0, 'AA': 0, 'CC': 52, 'BB': 351, 'HH': 506}
{'JJ': 3, 'FF': 4, 'GG': 5, 'EE': 3, 'DD': 2, 'II': 2, 'AA': 1, 'CC': 3, 'BB': 2, 'HH': 6}

{'FF': 0, 'GG': 0, 'EE': 63, 'DD': 440, 'II': 0, 'AA': 0, 'CC': 42, 'BB': 286, 'HH': 396}
{'FF': 6, 'GG': 7, 'EE': 5, 'DD': 4, 'II': 2, 'AA': 3, 'CC': 5, 'BB': 4, 'HH': 8}

{'GG': 0, 'FF': 0, 'EE': 60, 'II': 0, 'AA': 0, 'CC': 40, 'BB': 247, 'HH': 374}
{'GG': 4, 'FF': 3, 'EE': 2, 'II': 3, 'AA': 2, 'CC': 2, 'BB': 3, 'HH': 5}

{'GG': 0, 'FF': 0, 'EE': 39, 'II': 0, 'AA': 0, 'CC': 22, 'BB': 130}
{'GG': 2, 'FF': 3, 'EE': 4, 'II': 7, 'AA': 6, 'CC': 6, 'BB': 7}

{'GG': 0, 'FF': 0, 'EE': 18, 'II': 0, 'AA': 0, 'CC': 16}
{'GG': 6, 'FF': 5, 'EE': 4, 'II': 3, 'AA': 2, 'CC': 2}

{'GG': 0, 'FF': 0, 'II': 0, 'AA': 0, 'CC': 6}
{'GG': 3, 'FF': 2, 'II': 4, 'AA': 3, 'CC': 3}

{'FF': 0, 'AA': 0, 'II': 0}
{'FF': 4, 'AA': 3, 'II': 4}

[None, None, None, 'JJ', None, None, None, None, 'DD', None, None, None, None, None, 

### Part 2

In [12]:
# code for preparing and testing examples for part 2

example_dict_part_2 = {
    tuple(load_puzzle_input('example_1.txt')): "example_output"
}

In [13]:
# test part 2 solution

# test_code(part_2_solution, example_dict_part_2)